# Code sandbox

In [21]:
# Load modules
import os
import pandas as pd
import requests
import json
import csv
import time
from dateutil.relativedelta import relativedelta
import datetime as dt
from psaw import PushshiftAPI # https://github.com/dmarx/psaw


In [22]:
# Load modules
import pandas as pd
import requests
import json
import csv
import time
from dateutil.relativedelta import relativedelta
import datetime as dt
from psaw import PushshiftAPI # https://github.com/dmarx/psaw

## Submissions
def get_pages(subreddit: str, last_posttime = None):
    """Crawl a page of results from a given subreddit.
    :param subreddit: The subreddit to crawl.
    :param last_page: The last downloaded page.
    :return: A page of results.
    """

    url = "https://api.pushshift.io/reddit/search/submission"

    queries = {"subreddit": subreddit,\
               "size": 500,\
               "sort": "desc",\
               "sort_type": "created_utc"}

    # Called to "scroll down" page based on before
    if last_posttime is not None:
        queries["before"] = last_posttime

    results = requests.get(url, queries)

    if not results.ok:
        # something wrong happened
        raise Exception("Server returned status code {}".format(results.status_code))
    return results.json()["data"]


def crawl_subreddit(subreddit, max_submissions = 200000):
    """Crawl submissions from a subreddit.
    :param subreddit: The subreddit to crawl.
    :param max_submissions: The maximum number of submissions to download.
    :return: A list of submissions."""

    all_submissions = [] # empty list to hold all submissions
    last_posttime = None  # will become an empty list when reached the last page

    while len(all_submissions) < max_submissions:
        current_submissions = get_pages(subreddit, last_posttime)
        if len(current_submissions) == 0:
            break
        last_posttime = current_submissions[-1]["created_utc"]
        all_submissions += current_submissions

        #time.sleep(3)

        if len(all_submissions) % 10000 == 0: # to track progress for big pulls
            print(len(all_submissions))
    return all_submissions[:max_submissions]

## Comments
def crawl_comments(subreddit, before = None, after = None, max_comments = None):
    """Crawl comments from a subreddit
    :param subreddit: The subreddit to crawl.
    :param max_submissions: The max number of comments to download.
    :return: a data frame of comments"""

    api = PushshiftAPI()
    
    if before is not None and after is not None:
        gen = api.search_comments(subreddit = subreddit)
    else: 
        gen = api.search_comments(subreddit = subreddit,
                                 before = before,
                                 after = after)

    comments = []

    filename = "data/raw/comments" + "allcomments-" + str(today) + ".csv" # create filename

    counter = 0

    for c in gen:
        comments.append(c)
        counter += 1

        if counter % 100000 == 0:
            df = pd.DataFrame([obj.d_ for obj in comments])
            df.to_csv(filename, index = False, mode = "a")
            print(counter)

        # if len(comments) % 10000 == 0:
        #     print(len(comments))
         # Omit this to not limit to max_comments
        if max_comments is not None:
            if counter >= max_comments:
                 break
                    
    # Below code only used if the `if len(comments)` lines above not commented out
    #if False: # False flag - to be changed to True if we want to get rest of the results
    #    for c in gen:
    #        comments.append(c)


    # Create pandas data frame to return
    #df = pd.DataFrame([obj.d_ for obj in comments])

    return # empty return -- file saved


In [ ]:
today = dt.datetime.utcnow().date()
yesterday = today - dt.timedelta( = 1) # will count/collect posts after 00:00 on this date
two_yrs_ago = datetime.now() - relativedelta(years = 2)

crawl_comments("TheRedPill", before = today, after = two_yrs_ago)

## Check pull of entire subreddit / comments

In [21]:
import pandas as pd
import os

os.chdir("/Users/mariajoseherrera/Documents/Admin/yahb/Turing_Institute/trpred/data/raw/comments")# change wd

data = pd.read_csv("allcomments-asof-2020-05-17.csv", encoding = "UTF-8", header = None, names=list(range(40))) 

data

#with open('allcomments-asof-2020-05-17.csv', 'w') as csvfile:
  #  csvfile.write(not_csv)
    
    
# test_dict = {
#             "all_awardings": [],
#             "approved_at_utc": None,
#             "associated_award": None,
#             "author": "Limp-Entrance",
#             "author_flair_background_color": None,
#             "author_flair_css_class": None,
#             "author_flair_richtext": [],
#             "author_flair_template_id": None,
#             "author_flair_text": None,
#             "author_flair_text_color": None,
#             "author_flair_type": "text",
#             "author_fullname": "t2_59fgdzne",
#             "author_patreon_flair": False,
#             "author_premium": False,
#             "awarders": [],
#             "banned_at_utc": None,
#             "body": "It's worth if you need few flags to awake a hero.",
#             "can_mod_post": False,
#             "collapsed": False,
#             "collapsed_because_crowd_control": None,
#             "collapsed_reason": None,
#             "created_utc": 1589810052,
#             "distinguished": None,
#             "edited": False,
#             "gildings": {},
#             "id": "fr0z6zt",
#             "is_submitter": False,
#             "link_id": "t3_glvuns",
#             "locked": False,
#             "no_follow": True,
#             "parent_id": "t3_glvuns",
#             "permalink": "/r/TapTapHeroes/comments/glvuns/is_that_worth_buying/fr0z6zt/",
#             "retrieved_on": 1589810054,
#             "score": 1,
#             "send_replies": True,
#             "stickied": False,
#             "subreddit": "TapTapHeroes",
#             "subreddit_id": "t5_fodis",
#             "total_awards_received": 0,
#             "treatment_tags": []
#         }

# len(test_dict.keys()) # there are 40 fields


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39


In [9]:
import pandas as pd
import os

os.chdir("/Users/mariajoseherrera/Documents/Admin/yahb/Turing_Institute/trpred/data/raw/comments")# change wd
pd.read_json("dailycomments-2020-05-17.json") ## THIS WORKS!!!!

,all_awardings,associated_award,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received,treatment_tags
0,[],NaN,AskAboutMyDriving99,NaN,None,None,[],None,None,None,...,t1_fqxthon,/r/TheRedPill/comments/gl6i1v/it_always_gets_w...,1589764431,1,True,False,TheRedPill,t5_2ve1u,0,[]
1,[],NaN,redditor67tny,NaN,None,None,[],None,None,None,...,t1_fqxklmf,/r/TheRedPill/comments/glh1v0/what_holding_fra...,1589764028,3,True,False,TheRedPill,t5_2ve1u,0,[]
10,[],NaN,Pooddit,NaN,None,None,[],None,None,None,...,t3_glj61c,/r/TheRedPill/comments/glj61c/low_testosterone...,1589760115,1,True,False,TheRedPill,t5_2ve1u,0,[]
100,[],NaN,dino_10,NaN,None,None,[],None,None,None,...,t3_glk3we,/r/TheRedPill/comments/glk3we/holy_shit_guys_e...,1589740841,13,True,False,TheRedPill,t5_2ve1u,0,[]
101,[],NaN,AutoModerator,NaN,,moderator points,"[{'e': 'text', 't': 'mod'}]",None,mod,dark,...,t3_glk61y,/r/TheRedPill/comments/glk61y/elon_musk_is_red...,1589740740,1,False,False,TheRedPill,t5_2ve1u,0,[]
102,[],NaN,[deleted],NaN,,None,None,None,None,dark,...,t3_glijei,/r/TheRedPill/comments/glijei/youve_been_worki...,1589740728,1,True,False,TheRedPill,t5_2ve1u,0,[]
103,[],NaN,thernis,NaN,None,None,[],None,None,None,...,t1_fqxut5j,/r/TheRedPill/comments/gljfca/best_way_to_resp...,1589740661,3,True,False,TheRedPill,t5_2ve1u,0,[]
104,[],NaN,AutoModerator,NaN,,moderator points,"[{'e': 'text', 't': 'mod'}]",None,mod,dark,...,t3_glk3we,/r/TheRedPill/comments/glk3we/holy_shit_guys_e...,1589740501,1,False,True,TheRedPill,t5_2ve1u,0,[]
105,[],NaN,Post_shitter,NaN,None,None,[],None,None,None,...,t3_glj61c,/r/TheRedPill/comments/glj61c/low_testosterone...,1589740409,6,True,False,TheRedPill,t5_2ve1u,0,[]
106,[],NaN,AutoModerator,NaN,,moderator points,"[{'e': 'text', 't': 'mod'}]",None,mod,dark,...,t3_glk0rf,/r/TheRedPill/comments/glk0rf/holy_shit_guys_e...,1589740162,1,False,True,TheRedPill,t5_2ve1u,0,[]


## Automation code (daily)

In [6]:
# Calculate yesterday

today = dt.datetime.utcnow().date()

yesterday = today - dt.timedelta(days=1) # will count/collect posts after 00:00 on this date

# print(yesterday) # sanity check

### Submissions

#### Functions

In [7]:
## Submissions code
def get_pages(subreddit: str, last_posttime = None):
    """Crawl a page of results from a given subreddit over the past day.
    :param subreddit: The subreddit to crawl.
    :param last_posttime: The last downloaded page.
    :return: A page of results.
    """
    url = "https://api.pushshift.io/reddit/search/submission"
    
    # Calculate time window for past day
    today = dt.datetime.utcnow().date()
    yesterday = today - dt.timedelta(days = 1) # will count/collect posts after 00:00 on this date
    
    queries = {"subreddit": subreddit,\
               "size": 500,\
               "sort": "desc",\
               "sort_type": "created_utc",\
               "before": today,\
               "after": yesterday} 
    
    # Called to "scroll down" page based on before
    if last_posttime is not None:
        queries["before"] = last_posttime
    
    # Request data
    results = requests.get(url, params = queries)
    
    # Check for errors
    if not results.ok:
        # something wrong happened
        raise Exception("Server returned status code {}".format(results.status_code))
    
    return results.json()["data"]


def get_dailysubmissions(subreddit, max_submissions = 200000):
    """Crawl submissions from a subreddit over past day (hard coded in `get_pages()`).
    :param subreddit: The subreddit to crawl.
    :param max_submissions: The maximum number of submissions to download.
    :return: A list of submissions."""
    
    all_submissions = [] # empty list to hold all submissions
    last_posttime = None # will become an empty list when reached the last page
    
    while len(all_submissions) < max_submissions:
        current_submissions = get_pages(subreddit, last_posttime)
        if len(current_submissions) == 0:
            break
        last_posttime = current_submissions[-1]["created_utc"]
        all_submissions += current_submissions
        #time.sleep(3)
        if len(all_submissions) % 10000 == 0: # to track progress for big pulls
            print(len(all_submissions))
    return all_submissions[:max_submissions]


#### Test Code

In [8]:
# Test submissions
submissions = get_dailysubmissions("TheRedPill")

# Get date of submissions
yesterday = today - dt.timedelta(days = 1) # will count/collect posts after 00:00 on this date

# Save data as .json
os.chdir("/Users/mariajoseherrera/Documents/Admin/yahb/Turing Institute/trpred/data/raw/submissions")# change wd
filename = "submissions-" + str(yesterday) + ".json" # create filename 

with open(filename, 'w', encoding='utf-8') as f: # write file
    json.dump(submissions, f, ensure_ascii = False, indent=4)

### Comments

#### Functions

In [9]:
def get_dailycomments(subreddit, before, after, max_comments = 10000000):
    """Get comments from a subreddit over the past day
    :param subreddit: The subreddit to crawl.
    :param max_submissions: The max number of comments to download.
    :return: a data frame of comments""" 
    
    # Create instance of API
    api = PushshiftAPI() 
    
    # Using psaw package to access comments
    gen = api.search_comments(subreddit = subreddit,\
                              before = today,\
                             after = yesterday)    
    
    # Create empty container for comments
    comments = [] 
    
    for c in gen:
        comments.append(c) # append each item in generator
        
        if len(comments) % 10000 == 0: # track progress for large pulls
            print(len(comments))
         
        # Omit this to not limit to max_comments
#        if len(comments) >= max_comments:
#             break
    
    # Below code only used if the `if len(comments)` lines above not commented out
    if False: # False flag - to be changed to True if we want to get rest of the results
        for c in gen:
            comments.append(c)
     
    
    # Create pandas data frame containing comments to return        
    df = pd.DataFrame([obj.d_ for obj in comments])
    
    return df

In [10]:
# Get number of comments in past day (sanity check)
url = "https://api.pushshift.io/reddit/search/comment/"
queries = {"subreddit": "TheRedPill",\
            "size": 0,\
            "aggs" : "subreddit",\
            "after": yesterday,\
          "before": today} 

r = requests.get(url, params = queries)

# Get count (sanity check)
r.json()["aggs"]["subreddit"][0]["doc_count"]


283

In [14]:
# Calculate time window
today = dt.datetime.utcnow().date()
yesterday = today - dt.timedelta(days = 1) # will count/collect posts after 00:00 on this date


# Pull past day's comments (output: pandas df)
comments = get_dailycomments("TheRedPill", today, yesterday)


In [13]:
# Save data as .json
os.chdir("/Users/mariajoseherrera/Documents/Admin/yahb/Turing Institute/trpred/data/raw/comments")# change wd
filename = "comments-" + str(yesterday) + ".json" # create filename 

comments.to_json(filename)